In [1]:
import s3fs
import boto3
import botocore

import fastparquet
import pandas as pd
from urllib.parse import urlparse

from sqlalchemy import create_engine
import psycopg2 as pg
import io

import config

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline


There are a lot of files in the bucket, so rather than trying to load everything at once, download a few files to check out, figure out what data cleaning needs to be done,( and how to use parquet!)

In [2]:
# using fastparquet to convert to dataframe
test_data_file = './data/sample-data/2017-06-18/part-00099-6327ce66-f2d7-4bb6-b570-cc0db41e26e6.snappy.parquet'
pfile = fastparquet.ParquetFile(test_data_file)
df = pfile.to_pandas()

In [3]:
# first get a quick view of what data is there
df.head()

,articleId,channel,pageType,platform,publicationId,url,userId,timestamp
0,10639315,sport,article:news:readmore:poll|sim,nationals,2,http://www.mirror.co.uk/sport/football/transfe...,4f32564a-0503-4a8f-91ca-c980a7636d15,2017-06-18 02:31:02
1,10638305,news,article:news:vid|bc:readmore:readmore:readmore,nationals,4,http://www.dailyrecord.co.uk/news/uk-world-new...,ddb98cc7-086f-4fe0-8406-9eb0072324e4,2017-06-18 02:31:02
2,10636511,sport,article:news:readmore:readmore:poll|sim:vid|bc,nationals,2,http://www.mirror.co.uk/sport/football/news/ar...,b3d3e81b-c8c2-4533-a4a4-cea2892873e9,2017-06-18 02:31:02
3,4388389,news,article:news:vid|bc:poll|sim,nationals,2,http://www.mirror.co.uk/news/ampp3d/cats-9-liv...,876dca7b-d04f-4fe6-a51e-a1f909ec32b2,2017-06-18 02:31:02
4,10640509,news,article:news,nationals,2,http://www.mirror.co.uk/news/uk-news/google-me...,4961660a-c1cb-400a-ae8e-127877b431a5,2017-06-18 02:31:02


In [4]:
# the data looks fairly clean, but quick check to see if thee are any missing valeus:
df.isnull().values.any()

False

The following is to get a feel for what sort of data there are in the different fields:

In [5]:
# what are the differnt platforms the publications has?
df['platform'].describe()

count         22103
unique            3
top       nationals
freq          16270
Name: platform, dtype: object

In [6]:
df['platform'].value_counts()

nationals    16270
regionals     5577
ronionals      256
Name: platform, dtype: int64

In [7]:
# Channels: a lot of news as expected, and a lot of football...?
df['channel'].value_counts()

news                    9276
sport                   6499
3am                     3213
tv                      1237
whats-on                 524
lifestyle                410
science                  178
money                    172
tech                     141
arsenal-fc                73
showbiz                   66
chelsea-fc                54
entertainment             46
tottenham-hotspur-fc      46
ece_incoming              36
scotland-now              35
business                  29
play                      21
west-ham-united-fc        15
crystal-palace-fc         14
usvsth3m                   4
in-your-area               4
opinion                    2
special-features           2
home                       2
features                   1
config                     1
homes-and-property         1
motoring                   1
Name: channel, dtype: int64

In [8]:
# hmm, so what are ronionals...? could it be a typp?
roinials = [x for x in df['platform'] if x == 'ronionals']
print(len(roinials))

256


however, there is enough of them that it doesn't seem to be the case, and further clarification from the dataset told me that there are actually something called 'ronionals

In [10]:
# the timestamp column is obvious in what it means, but useful to check that it is indeed read as timestamp, 
# and yes it is
type(df['timestamp'][0])

pandas.tslib.Timestamp

A quick look at the range of times, although this is likely to be different -- eg during waking hours there are 
more visits etc

In [11]:
df['timestamp'].min()

Timestamp('2017-06-18 02:21:01')

In [12]:
df['timestamp'].max()

Timestamp('2017-06-18 04:39:01')

to have a unique id for publication and articles can help to look  comparisions between articles publications etc, 
and a quick look at the publicationId shows that different publications (from their urls) share the same publicationId. 

After a bit of clarification, the unique id for a publication is 'platform' + 'publicationId', and that for an article is 'platform' + 'articleId' -- so create those columns in the df

Also, the url can be split into 'host' and the last item in the path, which gives names to what the publication actually is and what is the article name-- would make a lot more sense to whoever is seeing the stats than ids, so extract those too

In [14]:
def get_article_name(url):
        path_arr = urlparse(url).path.split('/')
        return path_arr[len(path_arr) -1]
df['publicationSite'] = df['url'].apply(lambda x: urlparse(x).hostname)
df['articleUrlName'] = df['url'].apply(lambda x: get_article_name(x))
df['uniquePublicationId'] = df['platform'] + '-' + df['publicationId'].astype(str)
df['uniqueArticleId'] = df['platform'] + '-' + df['articleId'].astype(str)

In [15]:
# check that the new columns are created
df.head()


,articleId,channel,pageType,platform,publicationId,url,userId,timestamp,publicationSite,articleUrlName,uniquePublicationId,uniqueArticleId
0,10639315,sport,article:news:readmore:poll|sim,nationals,2,http://www.mirror.co.uk/sport/football/transfe...,4f32564a-0503-4a8f-91ca-c980a7636d15,2017-06-18 02:31:02,www.mirror.co.uk,arsenal-news-transfer-everton-berge-10639315,nationals-2,nationals-10639315
1,10638305,news,article:news:vid|bc:readmore:readmore:readmore,nationals,4,http://www.dailyrecord.co.uk/news/uk-world-new...,ddb98cc7-086f-4fe0-8406-9eb0072324e4,2017-06-18 02:31:02,www.dailyrecord.co.uk,heartbreaking-reason-dogs-being-deployed-10638305,nationals-4,nationals-10638305
2,10636511,sport,article:news:readmore:readmore:poll|sim:vid|bc,nationals,2,http://www.mirror.co.uk/sport/football/news/ar...,b3d3e81b-c8c2-4533-a4a4-cea2892873e9,2017-06-18 02:31:02,www.mirror.co.uk,arsenal-news-alexis-sanchez-ozil-10636511,nationals-2,nationals-10636511
3,4388389,news,article:news:vid|bc:poll|sim,nationals,2,http://www.mirror.co.uk/news/ampp3d/cats-9-liv...,876dca7b-d04f-4fe6-a51e-a1f909ec32b2,2017-06-18 02:31:02,www.mirror.co.uk,cats-9-lives-facts-behind-4388389,nationals-2,nationals-4388389
4,10640509,news,article:news,nationals,2,http://www.mirror.co.uk/news/uk-news/google-me...,4961660a-c1cb-400a-ae8e-127877b431a5,2017-06-18 02:31:02,www.mirror.co.uk,google-me-woman-whisked-feet-10640509,nationals-2,nationals-10640509


In [16]:
# lets have a quick look at data from a different day
pfile2 = fastparquet.ParquetFile('./data/sample-data/2017-06-16/part-00084-d05bf933-e313-4e4a-8aa2-01d8fe23de6f.snappy.parquet')
df2 = pfile2.to_pandas()

In [17]:
df2.head()

,articleId,channel,pageType,platform,publicationId,url,userId,timestamp
0,1887175,tv,article:news:gal,nationals,2,http://www.mirror.co.uk/tv/tv-news/olivia-colm...,c3c0fefd-48d5-4197-b389-955e3ef0e15b,2017-06-16 23:49:01
1,10636915,news,article:news:readmore:readmore,nationals,2,http://www.mirror.co.uk/news/uk-news/woman-sue...,fdd23106-8a87-4114-aca8-dab24adfd0ea,2017-06-16 23:49:01
2,13195705,news,article:news:readmore:readmore,regionals,2,http://www.manchestereveningnews.co.uk/news/gr...,cd1424c5-c333-4672-8e51-c8842318674f,2017-06-16 23:49:01
3,13194338,news,article:news:readmore:grid,regionals,47,http://www.glasgowlive.co.uk/news/glasgow-news...,d77f4835-ad85-443a-bbba-ab163f17cfaa,2017-06-16 23:49:01
4,10636161,sport,article:news,nationals,2,http://www.mirror.co.uk/sport/football/news/re...,e4eaa53e-758e-4c75-bb4b-1f10a3b00c73,2017-06-16 23:49:02


In [18]:
# yep, data doesn't have weird null values :-)
df.isnull().values.any()

False

In [77]:
df2.columns

Index(['articleId', 'channel', 'pageType', 'platform', 'publicationId', 'url',
       'userId', 'timestamp', 'publicationSite', 'articleUrlName',
       'uniquePublicationId', 'uniqueArticleId'],
      dtype='object')

questions I had for the data--  (most of these answered)
- what is the pageType column? how a user reached a particle page?
- how is publication id assigned? what is a 'normal' value? ie is there a lot of zeros etc? in which case really need to figure out how to get fastparquet to read them as strings!
- what is 'ronionals'...?!
- how are userIds assigned? does a real person only have 1 userId, or will different browser sesssions generate different userIds?

### data processing
So I got a feel for what the data looks like, its time to put it in a database that I am 
a bit more familiar with-- a bit of googling seems to indicate that you may be able to use parquet as a sort of db
but as I haven't used parquet at all before it seem easier to put it in a postgres db (easier to access from nodejs as well for the web app)

Postgres db as opposed to mongo (which I had worked a lot with before) since mongo has issues with aggregations on a lot of data being really slow... still have to try postgres properly but it should in theory be a bit faster...

so the processing work flow goes:
- read parquet file into df
- add extra columns (publicationSite, articleUrlName, uniquePublicationId, uniqueArticleId)
- save to postgres (I am using the google cloud psql, again for ease of access, also interesting to try out)

### Questions I want to get from the data, visuals:
- compare different publications do e.g. views over a week/day/hour? and is there a way to normalise the counts by e.g. number of visits divided by number of unique articles in the data (so that the larger publications don't dominated just because they have a lot more articles) -- small multiples could work nicely here
- user journeys: are there users who go to a lot of pages, and which categories of pages are they (e.g. are there distinct group of users, so that one group tend to go to news + science and the others to e.g only sports?) maybe flow chart? or some sort of venn diagram...?
- are there cateogories that are more popular at different times of the day? (e.g. more news visits in the morning, and tv in the evening...? could be traditional line charts, again small multiples could be useful
- also look for -- are there certain articles or publications that have a sudden surge in views in the time period-- these anomalies could indicate something interesting going on
- how are different publications linked by viewers? e.g. are certain publications likely to share users ?


### time to figure out how to connect to psql on gcloud:

There are good instructions on the cloud platform as to how to set one up
after creating the db instance and the db, 
create the table by :

```CREATE TABLE publication_logs ( articleId integer, channel text, pageType text, platform text, publicationId integer, url text, userId text, timestamp timestamp, publicationSite text, articleUrlName text, uniquePublicationId varchar(80), uniqueArticleId varchar(80) );```

which should let me write to it

first connect to the db instance using the gcloud proxy tool:
(https://cloud.google.com/sql/docs/postgres/connect-external-app)

`./cloud_sql_proxy -instances="data4democracy-wwymak-explore:us-central1:publications-experiment"=tcp:3306`

this would let me connect via sqlalchemy


In [20]:
address = address='postgresql://' + config.psql_user + ':' + config.psql_password + '@localhost:3306/userlogs'
engine = create_engine(address)
connection = engine.raw_connection()
cursor = connection.cursor()



In [87]:

#create Index column to use as primary key
# df2.reset_index(inplace=True)
# df2.rename(columns={'index':'Index'}, inplace =True)

#create the table but first drop if it already exists
# command = '''DROP TABLE IF EXISTS localytics_app2;
# CREATE TABLE localytics_app2
# (
# "Index" serial primary key,
# "Event" text,
# "Day" timestamp without time zone,
# );'''
# cursor.execute(command)
# connection.commit()

#stream the data using 'to_csv' and StringIO(); then use sql's 'copy_from' function
output = io.StringIO()
#ignore the index
df2.to_csv(output, sep='\t', header=False, index=False)
#jump to start of stream
output.seek(0)
contents = output.getvalue()
cur = connection.cursor()
#null values become ''
cur.copy_from(output, 'publication_logs', null="")    
connection.commit()
cur.close()

In [21]:
# I would want to pull data from s3, parse and write to psql without having to download it all, and boto3 is convenient
# for grabbing bucket contents,whereas s3fs is really handy for grabbing the parquet...

bucket = 'dataen-interview-data-dev'
access_key= config.aws_access_key
secret_key= config.aws_secret_key

client = boto3.client(
    's3',
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key
)
paginator2 = client.get_paginator('list_objects_v2')
Prefix='sample-data/2017-06-18/'

# Create a PageIterator from the Paginator
# page_iterator2 = paginator2.paginate(Bucket=bucket, Prefix='sample-data/2017-06-18/')

print(Prefix)
page_iterator2 = paginator2.paginate(Bucket=bucket, Prefix=Prefix)
keylist = []
for page in page_iterator2:
    keylist += page['Contents']
keys_to_parse = [x['Key'] for x in keylist if x['Size'] > 0]

# figured out how to parse parquet from s3:

def parse_file_to_df(object_key):
    s3 = s3fs.S3FileSystem(key=access_key, secret=secret_key)
    fs = s3fs.core.S3FileSystem(key=access_key, secret=secret_key)
    s3_path2 = bucket + '/' + object_key
    print(s3_path2)
    all_paths_from_s3 = fs.glob(path=s3_path2)
    myopen = s3.open
    #use s3fs as the filesystem
    fp_obj = fastparquet.ParquetFile(all_paths_from_s3,open_with=myopen)
    #convert to pandas dataframe
    df = fp_obj.to_pandas()
    print('df conversion')
    return df

sample-data/2017-06-18/


In [22]:
# and how to write to psql on remote host
def write_psql(df):
    output = io.StringIO()
    #ignore the index
    df.to_csv(output, sep='\t', header=False, index=False)
    print('to csv done')
    #jump to start of stream
    output.seek(0)
    contents = output.getvalue()
    connection = engine.raw_connection()
    cur = connection.cursor()
    #null values become ''
    print(cur)
    cur.copy_from(output, 'publication_logs', null="")
    connection.commit()
    cur.close()
    print('done')

after testing the above, and it seems to work fine, put it in a .py file and run from the command line like so:
`python parse-to-psql.py sample-data/2017-06-16/`

I have done part of 2017-06-16, 2017-06-17, 2017-06-18, except the `copy_from` seems to take a long time and my connection to the db hung up... :(

But there should be enough data in there to experiment with hopefully. 



In [123]:
# a previous experiment: it does work although in the middle of it python seems to be taking over my laptop's memory!
# the writing to psql seems to take a really long time?? 
#  see if I can parse a whole day of data into a df?
s3_path2 = bucket + '/' + 'sample-data/2017-06-18/part-*.snappy.parquet'
print(s3_path2)
all_paths_from_s3 = fs.glob(path=s3_path2)

myopen = s3.open
#use s3fs as the filesystem
fp_obj = fastparquet.ParquetFile(all_paths_from_s3,open_with=myopen)
#convert to pandas dataframe
df_remote_2 = fp_obj.to_pandas()

dataen-interview-data-dev/sample-data/2017-06-18/part-*.snappy.parquet


In [124]:
df_remote_2.shape

(6580063, 8)

In [126]:
df_remote_2.tail()

,articleId,channel,pageType,platform,publicationId,url,userId,timestamp
6580058,13197469,news,article:news:vid|bc:readmore:readmore:readmore,regionals,1,http://www.birminghammail.co.uk/news/midlands-...,57a6d9f3-421e-4a78-879d-99ce8944a4b7,2017-06-18 03:19:00
6580059,10639465,news,article:news:readmore:readmore,nationals,2,http://www.mirror.co.uk/news/real-life-stories...,4c7eb64b-cead-460d-be97-0e48e3df56c0,2017-06-18 03:19:00
6580060,10638932,sport,article:news:vid|bc:readmore:readmore:readmore...,nationals,4,http://www.dailyrecord.co.uk/sport/football/fo...,71435f93-a1ab-465b-9497-2b18a2c161a2,2017-06-18 03:19:00
6580061,13199066,sport,article:news:readmore:readmore,regionals,16,http://www.liverpoolecho.co.uk/sport/football/...,c19393ca-8e17-4cdc-aa3d-cbba6156bd45,2017-06-18 03:19:01
6580062,10640100,sport,article:news:readmore:vid|bc:poll|sim,nationals,2,http://www.mirror.co.uk/sport/football/news/re...,8dbb9ef2-7ad8-41de-a0b2-67f90b1bd5e3,2017-06-18 03:19:01
